In [1]:
%load_ext autoreload
%autoreload 2
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import processing_netcdf as pcdf
import geopandas as gpd
from geopandas import GeoDataFrame
import shapely.geometry 
import numpy as np
from shapely import geometry as gmty
from geofeather import to_geofeather, from_geofeather
import glob
import os
import pyarrow
from xclim import ensembles as ens
from xclim import subset
import matplotlib.pyplot as plt

In [2]:
folder = "/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/"

# Select and open Region (Shapefile)

In [3]:
######Select Region by uncommenting
#region = 'DDE_STF_20K_REG_FOR_VUE_S'
# region = 'DDE_STF_20K_UA_PER_VUE_S'
#region = 'DOM_BIO'
#region = 'REG_ECO'
#region = 'SDOM_BIO'
#region = 'Secteurs_Operations_Regionales'
#region = 'SREG_ECO'
region = 'territoire_guide'

In [4]:
shapefile = ("/home/mlopez/EXEC/Shapefiles/"+region+".shp")
poly0 = gpd.read_file(shapefile)
poly = poly0[poly0.geometry.type == 'Polygon'].to_crs('epsg:4326')
poly.head()

,TER_GUIDE,geometry
1,2a,"POLYGON ((-76.08748 46.47438, -76.08807 46.475..."
3,2c,"POLYGON ((-72.38130 45.65191, -72.38112 45.652..."
4,3ab,"POLYGON ((-79.06541 47.29977, -79.06474 47.299..."
5,3c,"POLYGON ((-72.11272 47.05779, -72.11182 47.058..."
6,3d,"POLYGON ((-69.62478 47.06785, -69.62615 47.066..."


## Import dataframe with polygons

In [5]:
dfpolyshape = from_geofeather('/home/mlopez/EXEC/Grids-polygons-regions/Grid-'+region+'.feather')
#dfpolyshape = from_geofeather('/home/mlopez/EXEC/Grids-polygons-regions/Grid-.feather')

  warnings.warn(



In [6]:
dfpolyshape.head()

,lat,lon,TER_GUIDE,geometry
0,52.957191,-67.712730,6opqr,"POLYGON ((-67.67107 52.93424, -67.67107 52.915..."
1,52.957191,-67.629402,6opqr,"POLYGON ((-67.67107 52.91553, -67.67107 52.934..."
2,52.957191,-67.546066,6opqr,"POLYGON ((-67.56730 52.91553, -67.57974 52.915..."
3,52.873859,-67.796059,6opqr,"POLYGON ((-67.75439 52.89329, -67.75439 52.832..."
4,52.873859,-67.712730,6opqr,"POLYGON ((-67.75439 52.83219, -67.75439 52.893..."


# Select variable

In [21]:
#Variable 
variable85="rcp85_tg_mean_annual"
variable45="rcp45_tg_mean_annual"
#variable="rcp[48]5_tg_mean_annual"

files85 = glob.glob(folder+"*"+variable85+".nc")
files45 = glob.glob(folder+"*"+variable45+".nc")
#ex: ACCESS1-3_rcp45_tn_mean_annual.nc
files85
files45

['/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/ACCESS1-3_rcp45_tg_mean_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/BNU-ESM_rcp45_tg_mean_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/CanESM2_rcp45_tg_mean_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/CMCC-CMS_rcp45_tg_mean_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/GFDL-ESM2M_rcp45_tg_mean_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/HadGEM2-CC_rcp45_tg_mean_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/INM-CM4_rcp45_tg_mean_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/IPSL-CM5A-LR_rcp45_tg_mean_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/IPSL-CM5B-LR_rcp45_tg_mean_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/MPI-ESM-LR_rcp45_tg_mean_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/NorESM1-M_rcp45_tg_mean_annual.nc']

# FOR RCP 45

## Subsetting arrays by periods of time and percentiles with Xclim

In [22]:
dsEns45= ens.create_ensemble(files45)
dsEns45

<xarray.Dataset>
Dimensions:      (lat: 320, lon: 416, realization: 11, time: 151)
Coordinates:
  * time         (time) datetime64[ns] 1950-01-01 1951-01-01 ... 2100-01-01
  * lat          (lat) float32 66.62331 66.53998 66.45665 ... 40.12437 40.04104
  * lon          (lon) float32 -89.04521 -88.96188 ... -54.54659 -54.46326
  * realization  (realization) int64 0 1 2 3 4 5 6 7 8 9 10
Data variables:
    tg_mean      (realization, time, lat, lon) float32 dask.array<chunksize=(1, 10, 320, 416), meta=np.ndarray>
Attributes:
    Conventions:     CF-1.5
    title:           ACCESS1-3 model output prepared for CMIP5 historical
    history:         CMIP5 compliant file produced from raw ACCESS model outp...
    institution:     CSIRO (Commonwealth Scientific and Industrial Research O...
    source:          ACCESS1-3 2011. Atmosphere: AGCM v1.0 (N96 grid-point, 1...
    redistribution:  Redistribution prohibited. For internal use only.

In [23]:
# Mapping data - 30 year means
tmp145 = dsEns45.sel(time=(dsEns45.time.dt.year>=1981))
window = 30
time145 = tmp145.time[0::window]

ds30yavg45 = tmp145.coarsen(time=window).mean()
ds30yavg45['time'] = time145
perc30yavg45 = ens.ensemble_percentiles(ds30yavg45)
perc30yavg45 # entire grid

# Mapping data

<xarray.Dataset>
Dimensions:      (lat: 320, lon: 416, realization: 11, time: 4)
Coordinates:
  * time         (time) datetime64[ns] 1981-01-01 2011-01-01 ... 2071-01-01
  * lat          (lat) float32 66.62331 66.53998 66.45665 ... 40.12437 40.04104
  * lon          (lon) float32 -89.04521 -88.96188 ... -54.54659 -54.46326
  * realization  (realization) int64 0 1 2 3 4 5 6 7 8 9 10
Data variables:
    tg_mean_p10  (time, lat, lon) float32 dask.array<chunksize=(1, 320, 416), meta=np.ndarray>
    tg_mean_p50  (time, lat, lon) float32 dask.array<chunksize=(1, 320, 416), meta=np.ndarray>
    tg_mean_p90  (time, lat, lon) float32 dask.array<chunksize=(1, 320, 416), meta=np.ndarray>

## Transform into DataFrame

In [24]:
%time df45 = perc30yavg45.drop('realization').to_dataframe().dropna()

df45

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)



CPU times: user 3min 5s, sys: 24min 15s, total: 27min 21s
Wall time: 28.8 s


tg_mean_p10  tg_mean_p50  tg_mean_p90
lat       lon        time                                             
66.623306 -89.045212 1981-01-01   259.535767   259.757019   259.995667
                     2011-01-01   260.684479   261.159607   261.800842
                     2041-01-01   261.750183   262.530670   263.948517
                     2071-01-01   262.040283   263.137421   264.745941
          -88.961884 1981-01-01   259.636353   259.858429   260.096039
...                                      ...          ...          ...
40.041039 -74.129143 2071-01-01   286.643341   287.808899   288.279480
          -74.045807 1981-01-01   285.045471   285.300079   285.372498
                     2011-01-01   285.980835   286.404816   286.786285
                     2041-01-01   286.491486   287.340820   287.734955
                     2071-01-01   286.677124   287.829163   288.284882

[304824 rows x 3 columns]

## Transform in Celsius and round

In [25]:
df45C = df45.copy()
df45C["tg_mean_p10"] = round(df45C["tg_mean_p10"] -273.15,2)
df45C["tg_mean_p50"] = round(df45C["tg_mean_p50"] -273.15,2)
df45C["tg_mean_p90"] = round(df45C["tg_mean_p90"] -273.15,2)

df45C

tg_mean_p10  tg_mean_p50  tg_mean_p90
lat       lon        time                                             
66.623306 -89.045212 1981-01-01       -13.61       -13.39       -13.15
                     2011-01-01       -12.47       -11.99       -11.35
                     2041-01-01       -11.40       -10.62        -9.20
                     2071-01-01       -11.11       -10.01        -8.40
          -88.961884 1981-01-01       -13.51       -13.29       -13.05
...                                      ...          ...          ...
40.041039 -74.129143 2071-01-01        13.49        14.66        15.13
          -74.045807 1981-01-01        11.90        12.15        12.22
                     2011-01-01        12.83        13.25        13.64
                     2041-01-01        13.34        14.19        14.58
                     2071-01-01        13.53        14.68        15.13

[304824 rows x 3 columns]

## Eliminate 2011, Pivot Table and Create new columns 

In [50]:
Region1i45 = df45C.reset_index()
print (Region1i45)
Region1i245 = Region1i45[Region1i45.time.dt.year!= 2011]
print ("2011 eliminated")
df45f = Region1i245.pivot_table(index=["lat","lon"], columns="time")
print ("DF pivoted")
df45f.columns = [year + "rcp45_tg_mean_p"+p
                     for p in ["10", "50", "90"]
                     for year in ["hist_","t2050_", "t2080_"]]
print("Columns renamed")
df45f

              lat        lon       time  tg_mean_p10  tg_mean_p50  tg_mean_p90
0       66.623306 -89.045212 1981-01-01       -13.61       -13.39       -13.15
1       66.623306 -89.045212 2011-01-01       -12.47       -11.99       -11.35
2       66.623306 -89.045212 2041-01-01       -11.40       -10.62        -9.20
3       66.623306 -89.045212 2071-01-01       -11.11       -10.01        -8.40
4       66.623306 -88.961884 1981-01-01       -13.51       -13.29       -13.05
...           ...        ...        ...          ...          ...          ...
304819  40.041039 -74.129143 2071-01-01        13.49        14.66        15.13
304820  40.041039 -74.045807 1981-01-01        11.90        12.15        12.22
304821  40.041039 -74.045807 2011-01-01        12.83        13.25        13.64
304822  40.041039 -74.045807 2041-01-01        13.34        14.19        14.58
304823  40.041039 -74.045807 2071-01-01        13.53        14.68        15.13

[304824 rows x 6 columns]
2011 eliminated
DF pivote

hist_rcp45_tg_mean_p10  t2050_rcp45_tg_mean_p10  \
lat       lon                                                           
40.041039 -89.045212                   11.50                    12.67   
          -88.961884                   11.48                    12.66   
          -88.878548                   11.47                    12.65   
          -88.795219                   11.45                    12.62   
          -88.711891                   11.46                    12.64   
...                                      ...                      ...   
66.623306 -61.629639                   -9.87                    -8.06   
          -61.546310                  -10.17                    -8.37   
          -61.462978                   -9.95                    -8.15   
          -61.379650                   -9.86                    -8.08   
          -61.296322                   -9.25                    -7.47   

                      t2080_rcp45_tg_mean_p10  hist_rcp45_tg_mean_p50  \
lat       lon                                                           
40.041039 -89.045212                    13.07                   11.64   
          -88.961884                    13.05                   11.62   
          -88.878548                    13.04                   11.61   
          -88.795219                    13.02                   11.59   
          -88.711891                    13.04                   11.61   
...                                       ...                     ...   
66.623306 -61.629639                    -8.27                   -9.65   
          -61.546310                    -8.57                   -9.94   
          -61.462978                    -8.36                   -9.73   
          -61.379650                    -8.28                   -9.65   
          -61.296322                    -7.67                   -9.05   

                      t2050_rcp45_tg_mean_p50  t2080_rcp45_tg_mean_p50  \
lat       lon                                                            
40.041039 -89.045212                    13.78                    14.34   
          -88.961884                    13.76                    14.32   
          -88.878548                    13.75                    14.32   
          -88.795219                    13.73                    14.30   
          -88.711891                    13.75                    14.32   
...                                       ...                      ...   
66.623306 -61.629639                    -7.34                    -6.49   
          -61.546310                    -7.63                    -6.79   
          -61.462978                    -7.40                    -6.57   
          -61.379650                    -7.31                    -6.49   
          -61.296322                    -6.70                    -5.88   

                      hist_rcp45_tg_mean_p90  t2050_rcp45_tg_mean_p90  \
lat       lon                                                           
40.041039 -89.045212                   11.67                    14.27   
          -88.961884                   11.65                    14.25   
          -88.878548                   11.64                    14.24   
          -88.795219                   11.62                    14.22   
          -88.711891                   11.64                    14.24   
...                                      ...                      ...   
66.623306 -61.629639                   -9.40                    -6.57   
          -61.546310                   -9.69                    -6.88   
          -61.462978                   -9.47                    -6.67   
          -61.379650                   -9.39                    -6.66   
          -61.296322                   -8.79                    -6.11   

                      t2080_rcp45_tg_mean_p90  
lat       lon                                  
40.041039 -89.045212                    14.75  
          -88.961884                    14.73  
          -88.878

# FOR RCP 85

## Subsetting arrays by periods of time and percentiles with Xclim

In [13]:
dsEns85= ens.create_ensemble(files85)
dsEns85

<xarray.Dataset>
Dimensions:      (lat: 320, lon: 416, realization: 11, time: 151)
Coordinates:
  * time         (time) datetime64[ns] 1950-01-01 1951-01-01 ... 2100-01-01
  * lat          (lat) float32 66.62331 66.53998 66.45665 ... 40.12437 40.04104
  * lon          (lon) float32 -89.04521 -88.96188 ... -54.54659 -54.46326
  * realization  (realization) int64 0 1 2 3 4 5 6 7 8 9 10
Data variables:
    tg_mean      (realization, time, lat, lon) float32 dask.array<chunksize=(1, 10, 320, 416), meta=np.ndarray>
Attributes:
    Conventions:     CF-1.5
    title:           ACCESS1-3 model output prepared for CMIP5 historical
    history:         CMIP5 compliant file produced from raw ACCESS model outp...
    institution:     CSIRO (Commonwealth Scientific and Industrial Research O...
    source:          ACCESS1-3 2011. Atmosphere: AGCM v1.0 (N96 grid-point, 1...
    redistribution:  Redistribution prohibited. For internal use only.

In [14]:
# Mapping data - 30 year means
tmp185 = dsEns85.sel(time=(dsEns85.time.dt.year>=1981))
window = 30
time185 = tmp185.time[0::window]

ds30yavg85 = tmp185.coarsen(time=window).mean()
ds30yavg85['time'] = time185
perc30yavg85 = ens.ensemble_percentiles(ds30yavg85)
perc30yavg85 # entire grid

<xarray.Dataset>
Dimensions:      (lat: 320, lon: 416, realization: 11, time: 4)
Coordinates:
  * time         (time) datetime64[ns] 1981-01-01 2011-01-01 ... 2071-01-01
  * lat          (lat) float32 66.62331 66.53998 66.45665 ... 40.12437 40.04104
  * lon          (lon) float32 -89.04521 -88.96188 ... -54.54659 -54.46326
  * realization  (realization) int64 0 1 2 3 4 5 6 7 8 9 10
Data variables:
    tg_mean_p10  (time, lat, lon) float32 dask.array<chunksize=(1, 320, 416), meta=np.ndarray>
    tg_mean_p50  (time, lat, lon) float32 dask.array<chunksize=(1, 320, 416), meta=np.ndarray>
    tg_mean_p90  (time, lat, lon) float32 dask.array<chunksize=(1, 320, 416), meta=np.ndarray>

## Transform into DataFrame

In [15]:
%time df85 = perc30yavg85.drop('realization').to_dataframe().dropna()
df85

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)



CPU times: user 3min 27s, sys: 26min 11s, total: 29min 38s
Wall time: 35.2 s


tg_mean_p10  tg_mean_p50  tg_mean_p90
lat       lon        time                                             
66.623306 -89.045212 1981-01-01   259.534363   259.676971   259.979248
                     2011-01-01   260.907867   261.137909   262.152130
                     2041-01-01   262.849518   263.773621   265.617798
                     2071-01-01   264.740723   267.102417   269.068146
          -88.961884 1981-01-01   259.635101   259.778412   260.079498
...                                      ...          ...          ...
40.041039 -74.129143 2071-01-01   288.501953   290.123566   290.468079
          -74.045807 1981-01-01   285.091705   285.309387   285.382385
                     2011-01-01   286.110199   286.495880   286.829041
                     2041-01-01   287.208649   288.221466   288.346680
                     2071-01-01   288.534546   290.137482   290.489441

[304824 rows x 3 columns]

## Transform in Celsius and round

In [16]:
df85C = df85.copy()
df85C["tg_mean_p10"] = round(df85C["tg_mean_p10"] -273.15,2)
df85C["tg_mean_p50"] = round(df85C["tg_mean_p50"] -273.15,2)
df85C["tg_mean_p90"] = round(df85C["tg_mean_p90"] -273.15,2)

df85C

tg_mean_p10  tg_mean_p50  tg_mean_p90
lat       lon        time                                             
66.623306 -89.045212 1981-01-01       -13.62       -13.47       -13.17
                     2011-01-01       -12.24       -12.01       -11.00
                     2041-01-01       -10.30        -9.38        -7.53
                     2071-01-01        -8.41        -6.05        -4.08
          -88.961884 1981-01-01       -13.51       -13.37       -13.07
...                                      ...          ...          ...
40.041039 -74.129143 2071-01-01        15.35        16.97        17.32
          -74.045807 1981-01-01        11.94        12.16        12.23
                     2011-01-01        12.96        13.35        13.68
                     2041-01-01        14.06        15.07        15.20
                     2071-01-01        15.38        16.99        17.34

[304824 rows x 3 columns]

## Eliminate 2011, Pivot Table and Create new columns 

In [51]:
Region1i85 = df85C.reset_index()
print (Region1i85)
Region1i285 = Region1i85[Region1i85.time.dt.year!= 2011]
print ("2011 eliminated")
df85f = Region1i285.pivot_table(index=["lat","lon"], columns="time")
print ("DF pivoted")
df85f.columns = [year + "rcp85_tg_mean_p"+p
                     for p in ["10", "50", "90"]
                     for year in ["hist_","t2050_", "t2080_"]]
print("Columns renamed")
df85f

              lat        lon       time  tg_mean_p10  tg_mean_p50  tg_mean_p90
0       66.623306 -89.045212 1981-01-01       -13.62       -13.47       -13.17
1       66.623306 -89.045212 2011-01-01       -12.24       -12.01       -11.00
2       66.623306 -89.045212 2041-01-01       -10.30        -9.38        -7.53
3       66.623306 -89.045212 2071-01-01        -8.41        -6.05        -4.08
4       66.623306 -88.961884 1981-01-01       -13.51       -13.37       -13.07
...           ...        ...        ...          ...          ...          ...
304819  40.041039 -74.129143 2071-01-01        15.35        16.97        17.32
304820  40.041039 -74.045807 1981-01-01        11.94        12.16        12.23
304821  40.041039 -74.045807 2011-01-01        12.96        13.35        13.68
304822  40.041039 -74.045807 2041-01-01        14.06        15.07        15.20
304823  40.041039 -74.045807 2071-01-01        15.38        16.99        17.34

[304824 rows x 6 columns]
2011 eliminated
DF pivote

hist_rcp85_tg_mean_p10  t2050_rcp85_tg_mean_p10  \
lat       lon                                                           
40.041039 -89.045212                   11.49                    13.51   
          -88.961884                   11.47                    13.49   
          -88.878548                   11.46                    13.48   
          -88.795219                   11.44                    13.46   
          -88.711891                   11.46                    13.48   
...                                      ...                      ...   
66.623306 -61.629639                   -9.89                    -7.19   
          -61.546310                  -10.20                    -7.49   
          -61.462978                  -10.00                    -7.28   
          -61.379650                   -9.93                    -7.20   
          -61.296322                   -9.33                    -6.60   

                      t2080_rcp85_tg_mean_p10  hist_rcp85_tg_mean_p50  \
lat       lon                                                           
40.041039 -89.045212                    14.80                   11.63   
          -88.961884                    14.78                   11.61   
          -88.878548                    14.77                   11.60   
          -88.795219                    14.75                   11.58   
          -88.711891                    14.77                   11.60   
...                                       ...                     ...   
66.623306 -61.629639                    -6.27                   -9.61   
          -61.546310                    -6.58                   -9.90   
          -61.462978                    -6.37                   -9.69   
          -61.379650                    -6.29                   -9.61   
          -61.296322                    -5.68                   -9.00   

                      t2050_rcp85_tg_mean_p50  t2080_rcp85_tg_mean_p50  \
lat       lon                                                            
40.041039 -89.045212                    14.69                    16.66   
          -88.961884                    14.67                    16.64   
          -88.878548                    14.66                    16.64   
          -88.795219                    14.64                    16.62   
          -88.711891                    14.65                    16.64   
...                                       ...                      ...   
66.623306 -61.629639                    -5.92                    -3.81   
          -61.546310                    -6.22                    -4.12   
          -61.462978                    -6.01                    -3.92   
          -61.379650                    -5.93                    -3.85   
          -61.296322                    -5.32                    -3.25   

                      hist_rcp85_tg_mean_p90  t2050_rcp85_tg_mean_p90  \
lat       lon                                                           
40.041039 -89.045212                   11.68                    14.88   
          -88.961884                   11.66                    14.86   
          -88.878548                   11.65                    14.85   
          -88.795219                   11.63                    14.83   
          -88.711891                   11.64                    14.85   
...                                      ...                      ...   
66.623306 -61.629639                   -9.39                    -5.20   
          -61.546310                   -9.68                    -5.57   
          -61.462978                   -9.46                    -5.43   
          -61.379650                   -9.38                    -5.42   
          -61.296322                   -8.77                    -4.87   

                      t2080_rcp85_tg_mean_p90  
lat       lon                                  
40.041039 -89.045212                    17.00  
          -88.961884                    16.98  
          -88.878

# Merge df45 and 85

In [54]:
dftg = pd.merge(df45f, df85f, on=["lat","lon"])
dftg.head()

hist_rcp45_tg_mean_p10  t2050_rcp45_tg_mean_p10  \
lat       lon                                                           
40.041039 -89.045212                   11.50                    12.67   
          -88.961884                   11.48                    12.66   
          -88.878548                   11.47                    12.65   
          -88.795219                   11.45                    12.62   
          -88.711891                   11.46                    12.64   

                      t2080_rcp45_tg_mean_p10  hist_rcp45_tg_mean_p50  \
lat       lon                                                           
40.041039 -89.045212                    13.07                   11.64   
          -88.961884                    13.05                   11.62   
          -88.878548                    13.04                   11.61   
          -88.795219                    13.02                   11.59   
          -88.711891                    13.04                   11.61   

                      t2050_rcp45_tg_mean_p50  t2080_rcp45_tg_mean_p50  \
lat       lon                                                            
40.041039 -89.045212                    13.78                    14.34   
          -88.961884                    13.76                    14.32   
          -88.878548                    13.75                    14.32   
          -88.795219                    13.73                    14.30   
          -88.711891                    13.75                    14.32   

                      hist_rcp45_tg_mean_p90  t2050_rcp45_tg_mean_p90  \
lat       lon                                                           
40.041039 -89.045212                   11.67                    14.27   
          -88.961884                   11.65                    14.25   
          -88.878548                   11.64                    14.24   
          -88.795219                   11.62                    14.22   
          -88.711891                   11.64                    14.24   

                      t2080_rcp45_tg_mean_p90  hist_rcp85_tg_mean_p10  \
lat       lon                                                           
40.041039 -89.045212                    14.75                   11.49   
          -88.961884                    14.73                   11.47   
          -88.878548                    14.72                   11.46   
          -88.795219                    14.70                   11.44   
          -88.711891                    14.72                   11.46   

                      t2050_rcp85_tg_mean_p10  t2080_rcp85_tg_mean_p10  \
lat       lon                                                            
40.041039 -89.045212                    13.51                    14.80   
          -88.961884                    13.49                    14.78   
          -88.878548                    13.48                    14.77   
          -88.795219                    13.46                    14.75   
          -88.711891                    13.48                    14.77   

                      hist_rcp85_tg_mean_p50  t2050_rcp85_tg_mean_p50  \
lat       lon                                                           
40.041039 -89.045212                   11.63                    14.69   
          -88.961884                   11.61                    14.67   
          -88.878548                   11.60                    14.66   
          -88.795219                   11.58                    14.64   
          -88.711891                   11.60                    14.65   

                      t2080_rcp85_tg_mean_p50  hist_rcp85_tg_mean_p90  \
lat       lon                                                           
40.041039 -89.045212                    16.66                   11.68   
          -88.961884                    16.64                   11.66   
          -88.878548                    16.64                   11.65   
          -88.795219                    16.62              

# Extract hist

In [55]:
dftg1 = dftg.drop(columns=["hist_rcp45_tg_mean_p10","hist_rcp45_tg_mean_p50", "hist_rcp45_tg_mean_p90", "hist_rcp85_tg_mean_p10", "hist_rcp85_tg_mean_p50", "hist_rcp85_tg_mean_p90"])
dftgh = dftg[["hist_rcp45_tg_mean_p50", "hist_rcp85_tg_mean_p50"]]

In [56]:
dftg1

t2050_rcp45_tg_mean_p10  t2080_rcp45_tg_mean_p10  \
lat       lon                                                            
40.041039 -89.045212                    12.67                    13.07   
          -88.961884                    12.66                    13.05   
          -88.878548                    12.65                    13.04   
          -88.795219                    12.62                    13.02   
          -88.711891                    12.64                    13.04   
...                                       ...                      ...   
66.623306 -61.629639                    -8.06                    -8.27   
          -61.546310                    -8.37                    -8.57   
          -61.462978                    -8.15                    -8.36   
          -61.379650                    -8.08                    -8.28   
          -61.296322                    -7.47                    -7.67   

                      t2050_rcp45_tg_mean_p50  t2080_rcp45_tg_mean_p50  \
lat       lon                                                            
40.041039 -89.045212                    13.78                    14.34   
          -88.961884                    13.76                    14.32   
          -88.878548                    13.75                    14.32   
          -88.795219                    13.73                    14.30   
          -88.711891                    13.75                    14.32   
...                                       ...                      ...   
66.623306 -61.629639                    -7.34                    -6.49   
          -61.546310                    -7.63                    -6.79   
          -61.462978                    -7.40                    -6.57   
          -61.379650                    -7.31                    -6.49   
          -61.296322                    -6.70                    -5.88   

                      t2050_rcp45_tg_mean_p90  t2080_rcp45_tg_mean_p90  \
lat       lon                                                            
40.041039 -89.045212                    14.27                    14.75   
          -88.961884                    14.25                    14.73   
          -88.878548                    14.24                    14.72   
          -88.795219                    14.22                    14.70   
          -88.711891                    14.24                    14.72   
...                                       ...                      ...   
66.623306 -61.629639                    -6.57                    -5.77   
          -61.546310                    -6.88                    -6.08   
          -61.462978                    -6.67                    -5.88   
          -61.379650                    -6.66                    -5.80   
          -61.296322                    -6.11                    -5.21   

                      t2050_rcp85_tg_mean_p10  t2080_rcp85_tg_mean_p10  \
lat       lon                                                            
40.041039 -89.045212                    13.51                    14.80   
          -88.961884                    13.49                    14.78   
          -88.878548                    13.48                    14.77   
          -88.795219                    13.46                    14.75   
          -88.711891                    13.48                    14.77   
...                                       ...                      ...   
66.623306 -61.629639                    -7.19                    -6.27   
          -61.546310                    -7.49                    -6.58   
          -61.462978                    -7.28                    -6.37   
          -61.379650                    -7.20                    -6.29   
          -61.296322                    -6.60                    -5.68   

                      t2050_rcp85_tg_mean_p50  t2080_rcp85_tg_mean_p50  \
lat       lon                                                            
40.041039 -89.045212  

# Merge both hist_50 and get the mean

In [36]:
dftgh["hist_tg_mean_p50"] = round((dftgh["hist_rcp45_tg_mean_p50"]+ dftgh["hist_rcp85_tg_mean_p50"])/2, 2)
dftgh2 = dftgh.reset_index()

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dftgh["hist_tg_mean_p50"] = round((dftgh["hist_rcp45_tg_mean_p50"]+ dftgh["hist_rcp85_tg_mean_p50"])/2, 2)



In [37]:
dftgh3 = dftgh2[["lat", "lon", "hist_tg_mean_p50"]]
dftgh3

,lat,lon,hist_tg_mean_p50
0,40.041039,-89.045212,11.64
1,40.041039,-88.961884,11.61
2,40.041039,-88.878548,11.60
3,40.041039,-88.795219,11.58
4,40.041039,-88.711891,11.60
...,...,...,...
76201,66.623306,-61.629639,-9.63
76202,66.623306,-61.546310,-9.92
76203,66.623306,-61.462978,-9.71
76204,66.623306,-61.379650,-9.63


# Merge means with the 45 and 85 DF

In [57]:
dftg_all = pd.merge(dftgh3, dftg1, on=["lat","lon"])

In [58]:
dftg_all

,lat,lon,hist_tg_mean_p50,t2050_rcp45_tg_mean_p10,t2080_rcp45_tg_mean_p10,t2050_rcp45_tg_mean_p50,t2080_rcp45_tg_mean_p50,t2050_rcp45_tg_mean_p90,t2080_rcp45_tg_mean_p90,t2050_rcp85_tg_mean_p10,t2080_rcp85_tg_mean_p10,t2050_rcp85_tg_mean_p50,t2080_rcp85_tg_mean_p50,t2050_rcp85_tg_mean_p90,t2080_rcp85_tg_mean_p90
0,40.041039,-89.045212,11.64,12.67,13.07,13.78,14.34,14.27,14.75,13.51,14.80,14.69,16.66,14.88,17.00
1,40.041039,-88.961884,11.61,12.66,13.05,13.76,14.32,14.25,14.73,13.49,14.78,14.67,16.64,14.86,16.98
2,40.041039,-88.878548,11.60,12.65,13.04,13.75,14.32,14.24,14.72,13.48,14.77,14.66,16.64,14.85,16.97
3,40.041039,-88.795219,11.58,12.62,13.02,13.73,14.30,14.22,14.70,13.46,14.75,14.64,16.62,14.83,16.95
4,40.041039,-88.711891,11.60,12.64,13.04,13.75,14.32,14.24,14.72,13.48,14.77,14.65,16.64,14.85,16.98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76201,66.623306,-61.629639,-9.63,-8.06,-8.27,-7.34,-6.49,-6.57,-5.77,-7.19,-6.27,-5.92,-3.81,-5.20,-2.56
76202,66.623306,-61.546310,-9.92,-8.37,-8.57,-7.63,-6.79,-6.88,-6.08,-7.49,-6.58,-6.22,-4.12,-5.57,-2.95
76203,66.623306,-61.462978,-9.71,-8.15,-8.36,-7.40,-6.57,-6.67,-5.88,-7.28,-6.37,-6.01,-3.92,-5.43,-2.82
76204,66.623306,-61.379650,-9.63,-8.08,-8.28,-7.31,-6.49,-6.66,-5.80,-7.20,-6.29,-5.93,-3.85,-5.42,-2.83


# Merge complete DF with Polygons

In [59]:
dftp = pd.merge(dftg_all, dfpolyshape, on=["lat","lon"])
print ("Merged with polygons")
dftp

Merged with polygons


,lat,lon,hist_tg_mean_p50,t2050_rcp45_tg_mean_p10,t2080_rcp45_tg_mean_p10,t2050_rcp45_tg_mean_p50,t2080_rcp45_tg_mean_p50,t2050_rcp45_tg_mean_p90,t2080_rcp45_tg_mean_p90,t2050_rcp85_tg_mean_p10,t2080_rcp85_tg_mean_p10,t2050_rcp85_tg_mean_p50,t2080_rcp85_tg_mean_p50,t2050_rcp85_tg_mean_p90,t2080_rcp85_tg_mean_p90,TER_GUIDE,geometry
0,44.957512,-74.629120,7.26,8.70,8.88,9.69,10.34,10.34,10.98,9.71,10.96,10.39,12.89,11.03,13.90,1a,"POLYGON ((-74.62590 44.99918, -74.58746 44.999..."
1,44.957512,-74.545792,7.23,8.67,8.85,9.67,10.32,10.31,10.96,9.68,10.93,10.36,12.87,11.00,13.87,1a,"POLYGON ((-74.58746 44.99876, -74.58746 44.999..."
2,44.957512,-74.462463,7.20,8.65,8.82,9.64,10.30,10.29,10.93,9.65,10.90,10.33,12.85,10.97,13.84,1a,"POLYGON ((-74.50413 44.99784, -74.50413 44.999..."
3,44.957512,-74.379128,7.19,8.63,8.80,9.62,10.29,10.27,10.91,9.63,10.88,10.32,12.83,10.96,13.83,1a,"POLYGON ((-74.42079 44.99530, -74.42079 44.999..."
4,44.957512,-74.295799,7.15,8.59,8.76,9.58,10.25,10.24,10.87,9.59,10.84,10.28,12.79,10.92,13.79,1a,"POLYGON ((-74.33746 44.99215, -74.33746 44.999..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16563,52.873859,-67.629402,-3.02,-1.52,-1.42,-0.71,0.09,0.52,1.19,-0.62,0.54,0.25,3.04,1.18,4.20,6opqr,"POLYGON ((-67.67107 52.83219, -67.67107 52.915..."
16564,52.873859,-67.546066,-2.96,-1.47,-1.37,-0.66,0.14,0.57,1.24,-0.57,0.59,0.30,3.09,1.23,4.25,6opqr,"POLYGON ((-67.58773 52.83219, -67.58773 52.904..."
16565,52.957191,-67.712730,-3.15,-1.66,-1.56,-0.85,-0.04,0.36,1.06,-0.76,0.40,0.12,2.91,1.05,4.07,6opqr,"POLYGON ((-67.67107 52.93424, -67.67107 52.915..."
16566,52.957191,-67.629402,-3.05,-1.56,-1.46,-0.75,0.06,0.46,1.15,-0.66,0.50,0.22,3.01,1.15,4.17,6opqr,"POLYGON ((-67.67107 52.91553, -67.67107 52.934..."


## Create GeoJson Files

In [61]:
listTG = []
for tg in dftp['TER_GUIDE'].unique().tolist():
    df2 = dftp[dftp.TER_GUIDE == tg]
    print (tg)
    listTG.append(df2)
listTG

1a
2c
3d
2b
2a
3c
3ab
4bc
4de
5ef
4f
4a
5jk
5bcd
5a
4gh
5hi
5g
6cdefg
6hi
6ab
6j
6mn
6kl
6opqr


[            lat        lon  hist_tg_mean_p50  t2050_rcp45_tg_mean_p10  \
 0     44.957512 -74.629120              7.26                     8.70   
 1     44.957512 -74.545792              7.23                     8.67   
 2     44.957512 -74.462463              7.20                     8.65   
 3     44.957512 -74.379128              7.19                     8.63   
 4     44.957512 -74.295799              7.15                     8.59   
 ...         ...        ...               ...                      ...   
 1236  46.207458 -72.629204              5.57                     7.08   
 1345  46.290791 -72.879189              5.43                     6.94   
 1347  46.290791 -72.795860              5.47                     6.99   
 1349  46.290791 -72.712532              5.49                     7.00   
 1351  46.290791 -72.629204              5.49                     7.00   
 
       t2080_rcp45_tg_mean_p10  t2050_rcp45_tg_mean_p50  \
 0                        8.88                     

In [62]:
for tg in listTG:
    geometry = tg["geometry"]
    crs = {'init': "epsg:4326"}
    gdf = GeoDataFrame(tg, crs=crs, geometry=geometry)
    #print (gdf['TER_GUIDE'].iloc[0])
    gdf.to_file(gdf['TER_GUIDE'].iloc[0]+"_tg_mean_annual.json", driver="GeoJSON")

# TODO: 
- Create same process for rcp 85
- Make procedure for historic time 